<a href="https://colab.research.google.com/github/duridudu/dupository/blob/%ED%98%BC%EA%B3%B5%EB%A8%B8%EC%8B%A0/%ED%98%BC%EA%B3%B5%EB%A8%B8%EC%8B%A0_chpater_04_1_%EB%A1%9C%EC%A7%80%EC%8A%A4%ED%8B%B1_%ED%9A%8C%EA%B7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**chapter 04. 다양한 분류 알고리즘**

##**럭키백의 확률**

* 럭키백에 들어갈 수 있는 생선은 7개. 사용할 수 있는 특성은 **길이, 높이, 두께, 대각선 길이, 무게**

* 럭키백에 들어갈 생선 7개의 확률을 출력해야 함. 

1. k-최근접 이웃을 써보자 : 이웃의 클래스 비율을 확률이라고 출력하면 되지 않을까?

2. 로지스틱 회귀로 7개의 생선 예측. 회귀 아니라 분류 모델이지만 계산한 값을 0~1 사이로 압축해서 확률로 보여줌.





###**K-최근접 이웃**
* 훈련셋과 테스트셋 표준화 전처리할 때 **훈련셋의 통계값으로 테스트셋을 변환해야함!**

* fish_target인 타깃 데이터에 7종의 생선이 있음. 
  
  ==> **다중 분류** : 타깃 데이터에 2개 이상의 클래스가 포함된 문제

  ==> 타깃값들은 사이킷런 모델에 전달되면 순서가 자동으로 알파벳 순으로 매겨짐. 
  
  KNeighborsClassifier 에서 정렬된 타깃값은 classes_ 속성에 저장되어 있음.





In [1]:
import pandas as pd
fish = pd.read_csv('https://bit.ly/fish_csv_data')
fish.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


In [2]:
print(pd.unique(fish['Species']))

['Bream' 'Roach' 'Whitefish' 'Parkki' 'Perch' 'Pike' 'Smelt']


In [6]:
fish_input=fish[['Weight','Length','Diagonal','Height','Width']].to_numpy()
fish_target=fish['Species'].to_numpy()

In [8]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(fish_input, fish_target, random_state=42)

#StandardScaler 클래스를 사용해 훈련셋과 테스트셋 표준화 전처리. 
##훈련셋의 통계값으로 테스트셋을 변환해야 함!! 
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
ss.fit(train_input) ##<--이걸로 테스트셋도 변환해야함!
train_scaled=ss.transform(train_input)
test_scaled=ss.transform(test_input)

#k-최근접 이웃 분류기의 확률 예측
from sklearn.neighbors import KNeighborsClassifier
kn=KNeighborsClassifier(n_neighbors=3)
kn.fit(train_scaled, train_target)
print(kn.score(train_scaled, train_target))
print(kn.score(test_scaled, test_target))


0.8907563025210085
0.85


In [9]:
print(kn.predict(test_scaled[:5])) #처음 5개 샘플의 타깃값

['Perch' 'Smelt' 'Pike' 'Perch' 'Perch']


In [10]:
import numpy as np
proba = kn.predict_proba(test_scaled[:5])
print(np.round(proba, decimals=4)) 
#첫번째 열이 bream, 두번째 열이 parkki에 대한 확률
#행은 n번째 샘플

[[0.     0.     1.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     1.     0.    ]
 [0.     0.     0.     1.     0.     0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]]


In [ ]:
#그러나! 3개의 최근접 이웃을 사용하기 때문에 확률은 항상 0/3, 1/3, 2/3, 3/3 중 하나일 것! 좀 이상함. 
#==> 로지스틱 회귀를 알아보자.

###**로지스틱 회귀-이진분류**
* 회귀이지만 분류 모델. 선형 방정식을 학습함. 

* 다중 회귀를 위한 선형 방정식과 같지만, **값이 아주 큰 음수일 때 0이 되고, 아주 큰 양수일 때 1이 됨** (확률이므로)

* sklearn.linear_model 하의 LogisticRegression 모듈

* 이진 분류 시 predict_proba()는 음성/양성 클래스에 대한 확률 출력,

  decision_function()는 양성 클래스에 대한 z값을 계산

In [11]:
#로지스틱 회귀로 이진 분류 수행하기 : 빙어와 도마 분류해보자
bream_smelt_indexes=(train_target == 'Bream') | (train_target=='Smelt') #도미와 빙어일 경우 True, 나머지는 False 들어감.
train_bream_smelt=train_scaled[bream_smelt_indexes]
target_bream_smelt=train_target[bream_smelt_indexes]

from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(train_bream_smelt, target_bream_smelt)

#훈련한 모델로 처음 5개 샘플 예측
print(lr.predict(train_bream_smelt[:5]))

['Bream' 'Smelt' 'Bream' 'Bream' 'Bream']


In [12]:
#이 샘플의 확률
#첫번째 열이 음성 클래스(0)에 대한 확률이고, 두번째 열이 양성 클래스(1)에 대한 확률
print(lr.predict_proba(train_bream_smelt[:5]))

[[0.99759855 0.00240145]
 [0.02735183 0.97264817]
 [0.99486072 0.00513928]
 [0.98584202 0.01415798]
 [0.99767269 0.00232731]]


In [14]:
#머가 양성이고 머가 음성 클래스?
print(lr.classes_)

['Bream' 'Smelt']


In [15]:
#이 로지스틱 회귀가 학습한 계수 확인
print(lr.coef_, lr.intercept_)
#z=-0.404wight-0.576length-0.663diagonal-1.013height-0.732width-2.161

[[-0.4037798  -0.57620209 -0.66280298 -1.01290277 -0.73168947]] [-2.16155132]


In [17]:
#로지스틱 회귀의 z값 출력
decisions=lr.decision_function(train_bream_smelt[:5])
print(decisions)

[-6.02927744  3.57123907 -5.26568906 -4.24321775 -6.0607117 ]


In [18]:
#이를 확률로 변환 ==> 양성 클래스에 대한 z값 반
from scipy.special import expit
print(expit(decisions))

[0.00240145 0.97264817 0.00513928 0.01415798 0.00232731]


###**로지스틱 회귀-다중분류**
* 이제 도미/빙어 2진 분류 말고 7개 종류 생선을 다 분류해보자.

* 학습 시 반복 횟수가 기본 100. 충분한 훈련을 위해 반복 횟수를 1000으로 늘려보자.

* 기본적으로 계수의 제곱을 규제함 like 릿지 회귀 --> by 매개변수 C **C가 작을수록 규제가 커짐 unlike alpha in 릿지

In [19]:
lr=LogisticRegression(C=20, max_iter=1000)
lr.fit(train_scaled, train_target)
print(lr.score(train_scaled, train_target))
print(lr.score(test_scaled, test_target))

#훈련셋, 테스트셋 점수 모두 높고 과대,과소적합으로 치우친 것 같지 X

0.9327731092436975
0.925


In [20]:
print(lr.predict(test_scaled[:5]))

['Perch' 'Smelt' 'Pike' 'Roach' 'Perch']


In [21]:
#위 샘플 5개의 확률 출력
proba=lr.predict_proba(test_scaled[:5])
print(np.round(proba, decimals=3))

[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]
 [0.011 0.034 0.306 0.007 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]


In [22]:
#위의 열들이 먼 순서냐? 바로 이 순서
print(lr.classes_)

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


In [23]:
#어떤 선형 방정식을 쓴걸까
print(lr.coef_.shape, lr.intercept_.shape)
#다중분류는 클래스마다 z값을 하나씩 계산하고 가장 높은 z값을 출력하는 클래스가 예측 클래스가 됨. 
#확률은 이진 분류의 시그모이드 함수와 다르게 '소프트맥스' 함수를 사용하여 7개의 z값을 확률로 변환함.
#소프트맥스 함수 : 여러 개의 선형 방정식의 출력값을 0~1 사이로 압축하고

(7, 5) (7,)


In [24]:
#소프트맥스 함수를 사용하여 확률로 바꿔보자
decision=lr.decision_function(test_scaled[:5])
print(np.round(decision, decimals=2))

[[ -6.5    1.03   5.16  -2.73   3.34   0.33  -0.63]
 [-10.86   1.93   4.77  -2.4    2.98   7.84  -4.26]
 [ -4.34  -6.23   3.17   6.49   2.36   2.42  -3.87]
 [ -0.68   0.45   2.65  -1.19   3.26  -5.75   1.26]
 [ -6.4   -1.99   5.82  -0.11   3.5   -0.11  -0.71]]


In [25]:
#소프트맥스 함수를 사용해보자. 위 배열을 소프트맥스에 전달
#axis는 소프트맥스를 계산할 축을 지정. 각 샘플에 대해 소프트맥스를 계산! 지정하지 않으면 배열 전체에 대해 소프트맥스를 계산한다.
from scipy.special import softmax
proba=softmax(decision, axis=1)
print(np.round(proba, decimals=3))


[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]
 [0.011 0.034 0.306 0.007 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]
